# * CEO Report Weekly

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

In [2]:
''' Input parameter '''

op_dir = 'data'
op_ceo_report_file = f'ceo_report_weekly_{str_curr_dt}'

v_start_date = 20250601
v_last_mth_date = 20251114
v_curr_date = 20251214

print(f'\nParameter input...\n')
print(f'   -> op_dir: {op_dir}')
print(f'   -> op_ceo_report_file: {op_ceo_report_file}')
print(f'\n   -> v_start_date: {v_start_date}')
print(f'   -> v_last_mth_date: {v_last_mth_date}')
print(f'   -> v_curr_date: {v_curr_date}')


Parameter input...

   -> op_dir: data
   -> op_ceo_report_file: ceo_report_weekly_20251218

   -> v_start_date: 20250601
   -> v_last_mth_date: 20251114
   -> v_curr_date: 20251214


## Import Transaction
-> DASH_PERFORMANCE

In [3]:
''' Execute transaction '''


# # Input parameter
# print(f'\nParameter input...')
# print(f'   -> v_start_date: {v_start_date}')
# print(f'   -> v_last_mth_date: {v_last_mth_date}')
# print(f'   -> v_curr_date: {v_curr_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()

query = (f"""
    WITH W_RAW_PRE_POST AS
    (
        SELECT TM_KEY_DAY, COMP_CD, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_DESC, ACTUAL, LOAD_DATE
        FROM GEOSPCAPPO.FCT_KPI_NEWCO_DIAMOND_ACTUAL
        WHERE METRIC_CD IN (
            'VIN00063' --Postpaid Subs Share (Subs) : TMH & DTAC
            , 'VIN00064' --Postpaid Subs Share (Subs) : AIS
            , 'VIN00066' --Postpaid Subs Share (Subs) : DTAC
            , 'VIN00065' --Postpaid Subs Share (Subs) : TMH
            , 'VIN00059' --Prepaid Subs Share (Subs) : TMH & DTAC
            , 'VIN00060' --Prepaid Subs Share (Subs) : AIS
            , 'VIN00062' --Prepaid Subs Share (Subs) : DTAC
            , 'VIN00061' --Prepaid Subs Share (Subs) : TMH
            )
        AND AREA_TYPE = 'HH'
        AND TM_KEY_DAY IN ({v_last_mth_date}, {v_curr_date})
    ) -->> W_RAWDATA
    -----------------------------------------------------------------------------------------------------------------------
    
    , W_RAW_MOBILE AS
    (
        -->> VIN00048	Mobile Subs Share (Subs) : TMH & DTAC
        SELECT TM_KEY_DAY
            , 'ALL' AS COMP_CD, 'VIN00048' AS METRIC_CD, 'Mobile Subs Share (Subs) : TMH & DTAC' AS METRIC_NAME
            , AREA_TYPE, AREA_CD, AREA_DESC
            , SUM(ACTUAL) ACTUAL, MAX(LOAD_DATE) LOAD_DATE
        FROM W_RAW_PRE_POST
        WHERE METRIC_CD IN (
            'VIN00063' --Postpaid Subs Share (Subs) : TMH & DTAC
            , 'VIN00059' --Prepaid Subs Share (Subs) : TMH & DTAC
            )
        GROUP BY TM_KEY_DAY, AREA_TYPE, AREA_CD, AREA_DESC
        
        UNION ALL
        
        -->> VIN00049	Mobile Subs Share (Subs) : AIS
        SELECT TM_KEY_DAY
            , 'ALL' AS COMP_CD, 'VIN00049' AS METRIC_CD, 'Mobile Subs Share (Subs) : AIS' AS METRIC_NAME
            , AREA_TYPE, AREA_CD, AREA_DESC
            , SUM(ACTUAL) ACTUAL, MAX(LOAD_DATE) LOAD_DATE
        FROM W_RAW_PRE_POST
        WHERE METRIC_CD IN (
            'VIN00064' --Postpaid Subs Share (Subs) : AIS
            , 'VIN00060' --Prepaid Subs Share (Subs) : AIS
            )
        GROUP BY TM_KEY_DAY, AREA_TYPE, AREA_CD, AREA_DESC
        
        UNION ALL
        
        -->> VIN00051	Mobile Subs Share (Subs) : DTAC
        SELECT TM_KEY_DAY
            , 'DTAC' AS COMP_CD, 'VIN00051' AS METRIC_CD, 'Mobile Subs Share (Subs) : DTAC' AS METRIC_NAME
            , AREA_TYPE, AREA_CD, AREA_DESC
            , SUM(ACTUAL) ACTUAL, MAX(LOAD_DATE) LOAD_DATE
        FROM W_RAW_PRE_POST
        WHERE METRIC_CD IN (
            'VIN00066' --Postpaid Subs Share (Subs) : DTAC
            , 'VIN00062' --Prepaid Subs Share (Subs) : DTAC
            )
        GROUP BY TM_KEY_DAY, AREA_TYPE, AREA_CD, AREA_DESC
        
        UNION ALL
        
        -->> VIN00050	Mobile Subs Share (Subs) : TMH
        SELECT TM_KEY_DAY
            , 'TRUE' AS COMP_CD, 'VIN00050' AS METRIC_CD, 'Mobile Subs Share (Subs) : TMH' AS METRIC_NAME
            , AREA_TYPE, AREA_CD, AREA_DESC
            , SUM(ACTUAL) ACTUAL, MAX(LOAD_DATE) LOAD_DATE
        FROM W_RAW_PRE_POST
        WHERE METRIC_CD IN (
            'VIN00065' --Postpaid Subs Share (Subs) : TMH
            , 'VIN00061' --Prepaid Subs Share (Subs) : TMH
            )
        GROUP BY TM_KEY_DAY, AREA_TYPE, AREA_CD, AREA_DESC
    ) -->> W_RAW_MOBILE
    -----------------------------------------------------------------------------------------------------------------------
    
    , W_ORG AS
    (
        SELECT DISTINCT ORGID_P, ZONE_TYPE, ORGID_G, TDS_SGMD, ORGID_H, HOP_HINT, ORGID_HH, D_CLUSTER
            , SUBSTR(CCAATT,1,2) AS PROVINCE_CD, PROVINCE_ENG
        FROM CDSAPPO.DIM_MOOC_AREA
        WHERE TEAM_CODE <> 'ไม่ระบุ' AND REMARK <> 'Dummy'
        AND SUBSTR(CCAATT,1,2) IN (10, 11, 12, 13) -->> 'Bangkok, Samut Prakan, Nonthaburi, Pathum Thani'
    ) -->> W_ORG
    -----------------------------------------------------------------------------------------------------------------------
    
    , W_SUBS_AGG AS
    (
        SELECT TM_KEY_DAY, COMP_CD, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_DESC, ACTUAL, LOAD_DATE
            , CASE 	WHEN METRIC_NAME LIKE 'Mobile%' THEN 'Mobile'
                    WHEN METRIC_NAME LIKE 'Prepaid%' THEN 'Prepaid'
                    WHEN METRIC_NAME LIKE 'Postpaid%' THEN 'Postpaid'
                    END PRODUCT
        FROM (
            SELECT TM_KEY_DAY, COMP_CD, METRIC_CD, METRIC_NAME
                , 'Z' AS AREA_TYPE, O.PROVINCE_CD AS AREA_CD, O.PROVINCE_ENG AS AREA_DESC
                , SUM(ACTUAL) ACTUAL, MAX(LOAD_DATE) LOAD_DATE
            FROM (
                SELECT * FROM W_RAW_PRE_POST
                UNION ALL
                SELECT * FROM W_RAW_MOBILE
            ) Z
            INNER JOIN W_ORG O
                ON O.ORGID_HH = Z.AREA_CD
            GROUP BY TM_KEY_DAY, COMP_CD, METRIC_CD, METRIC_NAME, O.PROVINCE_CD, O.PROVINCE_ENG
        ) TMP
    ) -->> W_SUBS_AGG
    -----------------------------------------------------------------------------------------------------------------------
    
    , W_MKS_AGG AS
    (
        SELECT *
        FROM (
            SELECT SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH
                , TM_KEY_DAY, COMP_CD, METRIC_CD, METRIC_NAME, PRODUCT, AREA_TYPE, AREA_CD, AREA_DESC, SUBS, TOTAL_SUBS, MKS, LOAD_DATE
                , LAG(MKS) OVER (PARTITION BY METRIC_CD, AREA_CD ORDER BY TM_KEY_DAY) AS PREV_MTD
                , MKS - LAG(MKS) OVER (PARTITION BY METRIC_CD, AREA_CD ORDER BY TM_KEY_DAY) AS MOM
                , SUBSTR(TM_KEY_DAY,1,6) AS MTH, SUBSTR(TM_KEY_DAY,7,2) AS DAY
            FROM (
                -->> Mobile MKS
                SELECT TM_KEY_DAY, COMP_CD, METRIC_CD, METRIC_NAME, PRODUCT, AREA_TYPE, AREA_CD, AREA_DESC
                    , ACTUAL AS SUBS
                    , SUM(CASE WHEN METRIC_CD IN ('VIN00049', 'VIN00050', 'VIN00051') THEN ACTUAL END) OVER (PARTITION BY TM_KEY_DAY, PRODUCT, AREA_CD) AS TOTAL_SUBS
                    , ACTUAL / SUM(CASE WHEN METRIC_CD IN ('VIN00049', 'VIN00050', 'VIN00051') THEN ACTUAL END) OVER (PARTITION BY TM_KEY_DAY, PRODUCT, AREA_CD) * 100 AS MKS
                    , LOAD_DATE
                FROM W_SUBS_AGG
                WHERE PRODUCT = 'Mobile'
                
                UNION ALL
                
                -->> Prepaid MKS
                SELECT TM_KEY_DAY, COMP_CD, METRIC_CD, METRIC_NAME, PRODUCT, AREA_TYPE, AREA_CD, AREA_DESC
                    , ACTUAL AS SUBS
                    , SUM(CASE WHEN METRIC_CD IN ('VIN00060', 'VIN00061', 'VIN00062') THEN ACTUAL END) OVER (PARTITION BY TM_KEY_DAY, PRODUCT, AREA_CD) AS TOTAL_SUBS
                    , ACTUAL / SUM(CASE WHEN METRIC_CD IN ('VIN00060', 'VIN00061', 'VIN00062') THEN ACTUAL END) OVER (PARTITION BY TM_KEY_DAY, PRODUCT, AREA_CD) * 100 AS MKS
                    , LOAD_DATE
                FROM W_SUBS_AGG
                WHERE PRODUCT = 'Prepaid'
                
                UNION ALL
                
                -->> Postpaid MKS
                SELECT TM_KEY_DAY, COMP_CD, METRIC_CD, METRIC_NAME, PRODUCT, AREA_TYPE, AREA_CD, AREA_DESC
                    , ACTUAL AS SUBS
                    , SUM(CASE WHEN METRIC_CD IN ('VIN00064', 'VIN00065', 'VIN00066') THEN ACTUAL END) OVER (PARTITION BY TM_KEY_DAY, PRODUCT, AREA_CD) AS TOTAL_SUBS
                    , ACTUAL / SUM(CASE WHEN METRIC_CD IN ('VIN00064', 'VIN00065', 'VIN00066') THEN ACTUAL END) OVER (PARTITION BY TM_KEY_DAY, PRODUCT, AREA_CD) * 100 AS MKS
                    , LOAD_DATE
                FROM W_SUBS_AGG
                WHERE PRODUCT = 'Postpaid'
            ) T1
        ) T2
        WHERE TM_KEY_DAY = {v_curr_date}
    ) -->> W_MKS_AGG
    -----------------------------------------------------------------------------------------------------------------------
    -----------------------------------------------------------------------------------------------------------------------
    
    
    /* Adhoc Data */


    SELECT /*+PARALLEL(8)*/ A.*
        , T.DAYS_IN_MONTH, T.PERIODFLAG
    FROM (
        -->> Mobile MKS : Z level
        SELECT 
            NULL AS TM_KEY_YR, NULL AS TM_KEY_QTR, CAST(TM_KEY_MTH AS INT) AS TM_KEY_MTH, NULL AS TM_KEY_WK, TM_KEY_DAY
            , CASE 	WHEN (NOT TRUNC(CAST(TO_TIMESTAMP(TO_NCHAR(TM_KEY_DAY),'YYYYMMDD') AS DATE)) IS NULL) THEN TRUNC(CAST(TO_TIMESTAMP(TO_NCHAR(TM_KEY_DAY),'YYYYMMDD') AS DATE)) 
                    WHEN (NOT TRUNC(TO_DATE(TO_NCHAR(TM_KEY_DAY),'YYYY-MM-DD')) IS NULL) THEN TRUNC(TO_DATE(TO_NCHAR(TM_KEY_DAY),'YYYY-MM-DD')) 
                    ELSE NULL END DYNAMIC_DAY
            , COMP_CD--, METRIC_CD, METRIC_NAME
            , CASE  WHEN METRIC_CD = 'VIN00048' THEN 'VIN00011'
                    WHEN METRIC_CD = 'VIN00049' THEN 'VIN00012'
                    WHEN METRIC_CD = 'VIN00050' THEN 'VIN00013'
                    WHEN METRIC_CD = 'VIN00051' THEN 'VIN00014'
                    WHEN METRIC_CD = 'VIN00059' THEN 'VIN00029'
                    WHEN METRIC_CD = 'VIN00060' THEN 'VIN00030'
                    WHEN METRIC_CD = 'VIN00061' THEN 'VIN00031'
                    WHEN METRIC_CD = 'VIN00062' THEN 'VIN00032'
                    WHEN METRIC_CD = 'VIN00063' THEN 'VIN00037'
                    WHEN METRIC_CD = 'VIN00064' THEN 'VIN00038'
                    WHEN METRIC_CD = 'VIN00065' THEN 'VIN00039'
                    WHEN METRIC_CD = 'VIN00066' THEN 'VIN00040'
                    END METRIC_CD
            , CASE  WHEN METRIC_CD = 'VIN00048' THEN 'Mobile Subs Share : TMH & DTAC'
                    WHEN METRIC_CD = 'VIN00049' THEN 'Mobile Subs Share : AIS'
                    WHEN METRIC_CD = 'VIN00050' THEN 'Mobile Subs Share : TMH'
                    WHEN METRIC_CD = 'VIN00051' THEN 'Mobile Subs Share : DTAC'
                    WHEN METRIC_CD = 'VIN00059' THEN 'Prepaid Subs Share : TMH & DTAC'
                    WHEN METRIC_CD = 'VIN00060' THEN 'Prepaid Subs Share : AIS'
                    WHEN METRIC_CD = 'VIN00061' THEN 'Prepaid Subs Share : TMH'
                    WHEN METRIC_CD = 'VIN00062' THEN 'Prepaid Subs Share : DTAC'
                    WHEN METRIC_CD = 'VIN00063' THEN 'Postpaid Subs Share : TMH & DTAC'
                    WHEN METRIC_CD = 'VIN00064' THEN 'Postpaid Subs Share : AIS'
                    WHEN METRIC_CD = 'VIN00065' THEN 'Postpaid Subs Share : TMH'
                    WHEN METRIC_CD = 'VIN00066' THEN 'Postpaid Subs Share : DTAC'
                    END METRIC_NAME
            , NULL AS METRIC_NAME_GROUP, NULL AS ACTUAL_AS_OF, NULL AS AGG_TYPE
            , MKS AS ACTUAL_SNAP
            , NULL AS TARGET_SNAP, NULL AS BASELINE_SNAP, NULL AS ACH_SNAP, NULL AS GAP_SNAP
            , MKS AS ACTUAL_AGG_MTH
            , NULL AS TARGET_AGG_MTH, NULL AS ACH_AGG_MTH, NULL AS GAP_AGG_MTH, NULL AS ACTUAL_AGG_YR, NULL AS TARGET_AGG_YR, NULL AS ACH_AGG_YR, NULL AS GAP_AGG_YR, NULL AS WOW, NULL AS WOW_PERCENT
            , MOM
            , NULL AS MOM_PERCENT, NULL AS QOQ, NULL AS QOQ_PERCENT, NULL AS MTD_YOY, NULL AS MTD_YOY_PERCENT, NULL AS CENTER, NULL AS METRIC_GRP
            , CASE 	WHEN METRIC_CD IN ('VIN00048', 'VIN00049', 'VIN00050', 'VIN00051') THEN 'Mobile' 
                    WHEN METRIC_CD IN ('VIN00059', 'VIN00060', 'VIN00061', 'VIN00062') THEN 'Prepaid' 
                    WHEN METRIC_CD IN ('VIN00063', 'VIN00064', 'VIN00065', 'VIN00066') THEN 'Postpaid' 
                    END PRODUCT_GRP
            , AREA_TYPE, AREA_CD, AREA_DESC AS AREA_NAME
            , LOAD_DATE AS PPN_TM
        --	, NULL AS WTD_YOY, NULL AS WTD_YOY_PERCENT, NULL AS MTD_YOY, NULL AS MTD_YOY_PERCENT, NULL AS QTD_YOY, NULL AS QTD_YOY_PERCENT, NULL AS YTD_YOY, NULL AS YTD_YOY_PERCENT, NULL AS P2W, NULL AS P2M, NULL AS P2Q, NULL AS P2W_PERCENT, NULL AS P2M_PERCENT, NULL AS P2Q_PERCENT
        FROM W_MKS_AGG
        
        UNION ALL
        
        -->> CEO Report Data
        
        SELECT TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY
            , CASE 	WHEN (NOT TRUNC(CAST(TO_TIMESTAMP(TO_NCHAR(TM_KEY_DAY),'YYYYMMDD') AS DATE)) IS NULL) THEN TRUNC(CAST(TO_TIMESTAMP(TO_NCHAR(TM_KEY_DAY),'YYYYMMDD') AS DATE)) 
                    WHEN (NOT TRUNC(TO_DATE(TO_NCHAR(TM_KEY_DAY),'YYYY-MM-DD')) IS NULL) THEN TRUNC(TO_DATE(TO_NCHAR(TM_KEY_DAY),'YYYY-MM-DD')) 
                    ELSE NULL END DYNAMIC_DAY
            , COMP_CD, METRIC_CD, METRIC_NAME, METRIC_NAME_GROUP, ACTUAL_AS_OF, AGG_TYPE
            , ACTUAL_SNAP, TARGET_SNAP, BASELINE_SNAP, ACH_SNAP, GAP_SNAP
            , ACTUAL_AGG_MTH, TARGET_AGG_MTH, ACH_AGG_MTH, GAP_AGG_MTH
            , ACTUAL_AGG_YR, TARGET_AGG_YR, ACH_AGG_YR, GAP_AGG_YR
            , WOW, WOW_PERCENT, MOM, MOM_PERCENT, QOQ, QOQ_PERCENT, MTD_YOY, MTD_YOY_PERCENT--, YOY, YOY_PERCENT
            , CENTER, METRIC_GRP, PRODUCT_GRP, AREA_TYPE, AREA_CD, AREA_NAME, PPN_TM
        --	, WTD_YOY, WTD_YOY_PERCENT, MTD_YOY, MTD_YOY_PERCENT, QTD_YOY, QTD_YOY_PERCENT, YTD_YOY, YTD_YOY_PERCENT, P2W, P2M, P2Q, P2W_PERCENT, P2M_PERCENT, P2Q_PERCENT
        FROM GEOSPCAPPO.AGG_PERF_NEWCO
        WHERE ((
            METRIC_CD IN (
            -->> Revenue
            'B2R010100' --Postpaid Revenue B2C
            , 'B1R000100' --Prepaid Revenue
            , 'TB3R000100' --TOL Revenue
            , 'TB4R000100' --TVS Revenue
            -->> Subs
            , 'B2S010600' --Postpaid Reported SubBase B2C
            , 'DB2S010600' --Postpaid Reported SubBase B2C : DTAC
            , 'TB2S010600' --Postpaid Reported SubBase B2C : TMH
            , 'B1S000600' --Prepaid Active Caller 30D Rolling
            , 'DB1S000600' --Prepaid Active Caller 30D Rolling : DTAC
            , 'TB1S000600' --Prepaid Active Caller 30D Rolling : TMH
            , 'B1S000601' --Prepaid Active Caller 30D Rolling : Thai Mass
            , 'B1S000602' --Prepaid Active Caller 30D Rolling : AEC/Migrants
            , 'B1S000603' --Prepaid Active Caller 30D Rolling : Tourists
            , 'TB3S000600' --FTTx Reported SubBase
            , 'TB4S000500' --TVS Active Subs
            , 'TNSC00150' --TrueID TV Box MAU
            -->> Churn
            , 'B2S010200' --Postpaid Churn Subs B2C
            , 'B2S010201' --Postpaid Churn Subs Voluntary B2C
            , 'B2S010202' --Postpaid Churn Subs Involuntary B2C
            , 'B1S000200' --Prepaid Churn Subs
            , 'TSER13100' --FTTx Churn Subs
            , 'TSER13102' --FTTx Churn Subs Voluntary
            , 'TSER13103' --FTTx Churn Subs Involuntary
            -->> GA
            , 'B2S010100CS' --Postpaid Gross Adds B2C
            , 'B1S000101CS' --Prepaid Gross Adds
            , 'TB3S000102CS' --TOL Gross Adds Connected : Consumer
            , 'TB4S001400CS' --TVS Now Gross Adds
            , 'B2S010100CG' --Postpaid Gross Adds B2C - GEO Channel
            , 'B1S000101CG' --Prepaid Gross Adds - GEO Channel
            , 'TB3S000102CG' --TOL Gross Adds Connected : Consumer - GEO Channel
            , 'TB4S001400CG' --TVS Now Gross Adds - GEO Channel
            -->> Inflow
            , 'B2R010500CS' --Postpaid Inflow M1 B2C
            , 'B1R000900CS' --Prepaid Inflow M1
            , 'TB3R000601CS' --TOL Inflow M1 Connected : Consumer
            , 'TB4R001700CS' --TVS Now Inflow M1
            , 'B2R010500CG' --Postpaid Inflow M1 B2C - GEO Channel
            , 'B1R000900CG' --Prepaid Inflow M1 - GEO Channel
            , 'TB3R000601CG' --TOL Inflow M1 Connected : Consumer - GEO Channel
            , 'TB4R001700CG' --TVS Now Inflow M1 - GEO Channel
            -- >> MKS
            , 'VIN00080','VIN00081','VIN00082','VIN00083','VIN00084' --Broadband MKS
        --	, 'VIN00011','VIN00012','VIN00013','VIN00014','VIN00029','VIN00030','VIN00031','VIN00032','VIN00037','VIN00038','VIN00039','VIN00040', --Mobile MKS
                )
            AND (AREA_TYPE IN ('P','G','Z') OR AREA_CD IN ('906103', '906112', '906102'))
            )
        OR (
            METRIC_CD IN ('VIN00011','VIN00012','VIN00013','VIN00014','VIN00029','VIN00030','VIN00031','VIN00032','VIN00037','VIN00038','VIN00039','VIN00040') --Mobile MKS
            AND (AREA_CD IN ('P', 'BMA', 'UPC', 'EAST', '906103', '906112', '906102') OR AREA_TYPE = 'G')
            )
        )
        AND TM_KEY_DAY >= {v_start_date}
    ) A

    LEFT JOIN GEOSPCAPPO.DIM_TIME_CITRINE T
        ON T.TM_KEY_DAY = A.TM_KEY_DAY
""")


try:
    # Create Dataframe
    src_cur.execute(query)
    rows = src_cur.fetchall()
    src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\nDataFrame: {src_df.shape[0]} rows, {src_df.shape[1]} columns')

    # Generate CSV file
    src_df.to_csv(f'{op_dir}/{op_ceo_report_file}.csv', index=False, encoding='utf-8')
    print(f'\n   -> Generate "{op_ceo_report_file}.csv" successfully')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Data as of 2025-12-18, 11:01:52

DataFrame: 111248 rows, 42 columns

   -> Generate "ceo_report_weekly_20251218.csv" successfully


## Review

In [4]:
src_df

,TM_KEY_YR,TM_KEY_QTR,TM_KEY_MTH,TM_KEY_WK,TM_KEY_DAY,DYNAMIC_DAY,COMP_CD,METRIC_CD,METRIC_NAME,METRIC_NAME_GROUP,...,MTD_YOY_PERCENT,CENTER,METRIC_GRP,PRODUCT_GRP,AREA_TYPE,AREA_CD,AREA_NAME,PPN_TM,DAYS_IN_MONTH,PERIODFLAG
0,NaN,NaN,202512,NaN,20251214,2025-12-14,ALL,VIN00011,Mobile Subs Share : TMH & DTAC,None,...,NaN,None,None,Mobile,Z,10,Bangkok,2025-12-16 20:17:41,31,N
1,NaN,NaN,202512,NaN,20251214,2025-12-14,ALL,VIN00011,Mobile Subs Share : TMH & DTAC,None,...,NaN,None,None,Mobile,Z,11,Samut Prakan,2025-12-16 20:17:41,31,N
2,NaN,NaN,202512,NaN,20251214,2025-12-14,ALL,VIN00011,Mobile Subs Share : TMH & DTAC,None,...,NaN,None,None,Mobile,Z,12,Nonthaburi,2025-12-16 20:17:41,31,N
3,NaN,NaN,202512,NaN,20251214,2025-12-14,ALL,VIN00011,Mobile Subs Share : TMH & DTAC,None,...,NaN,None,None,Mobile,Z,13,Pathum Thani,2025-12-16 20:17:41,31,N
4,NaN,NaN,202512,NaN,20251214,2025-12-14,ALL,VIN00012,Mobile Subs Share : AIS,None,...,NaN,None,None,Mobile,Z,10,Bangkok,2025-12-16 20:17:41,31,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111243,2025.0,20253.0,202507,2025030.0,20250724,2025-07-24,ALL,VIN00030,Prepaid Subs Share : AIS,Prepaid Subs Share,...,2.992530,Revenue,Market Share,Prepaid,G,GX1,BMA : West,2025-12-18 02:41:30,31,N
111244,2025.0,20253.0,202507,2025030.0,20250725,2025-07-25,ALL,VIN00030,Prepaid Subs Share : AIS,Prepaid Subs Share,...,2.977100,Revenue,Market Share,Prepaid,G,GX1,BMA : West,2025-12-18 02:41:30,31,N
111245,2025.0,20253.0,202507,2025030.0,20250726,2025-07-26,ALL,VIN00030,Prepaid Subs Share : AIS,Prepaid Subs Share,...,3.024697,Revenue,Market Share,Prepaid,G,GX1,BMA : West,2025-12-18 02:41:30,31,N
111246,2025.0,20253.0,202507,2025030.0,20250727,2025-07-27,ALL,VIN00030,Prepaid Subs Share : AIS,Prepaid Subs Share,...,3.041741,Revenue,Market Share,Prepaid,G,GX1,BMA : West,2025-12-18 02:41:30,31,N
